In [1]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [ ]:
install_requirements()

In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

In [4]:
setup_env()

GROQ_API_KEY is set.


In [5]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union

DEFAULT_MODEL = "llama-3.3-70b-versatile"


class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))

In [6]:
from typing import List, Union, Any
from pydantic import BaseModel
from groq import Groq


class UserStory(BaseModel):
    """Represents a User Story"""

    title: str
    description: str
    acceptance_criteria: List[str]
    story_points: int


class UserStories(BaseModel):
    """Represents a collection of User Stories"""

    user_stories: List[UserStory]


class PRD(BaseModel):
    """Represents a Product Requirements Document (PRD)"""

    title: str
    overview: str
    problem_statement: str
    business_domains: List[str]
    proposed_solution: str
    target_audience: str
    features: List[str]
    constraints: List[str]
    assumptions: List[str]
    dependencies: List[str]
    user_personas: List[str]
    conclusion: str


class MiniPM:
    """An AI agent that acts as a Product Manager to help project managers create PRD and User Stories for their projects."""

    llm: Groq
    project_description: str

    def __init__(self, project_description: str):
        """Initializes the MiniPM AI agent with the project description."""
        self.project_description = project_description
        self.llm = get_groq_client()

    def get_prd(
        self, response_format="markdown", augment=True
    ) -> Union[PRD, LLMErrorResponse, Any]:
        """Gets the Product Requirements Document (PRD) for the project."""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create a Product Requirements Document (PRD) for their projects.
                """
        prompt = f"""
                Provide a Product Requirements Document (PRD) for the project with the following details:
                Details: {self.project_description}
                """
        response = llm(prompt=prompt, response_model=PRD, system=system)

        if augment:
            response = self._augment_prd(response)

        if response_format == "json":
            return response.model_dump_json()
        elif response_format == "markdown":
            response_dict = response.model_dump()
            keys = response_dict.keys()
            markdown = f"# Product Requirements Document (PRD) for {response.title}\n"
            for key in keys:
                title = key.title().replace("_", " ").replace("Prd", "PRD")
                if key == "title":
                    title = ""
                    continue
                markdown += f"## {title}\n"
                value = response_dict[key]
                if type(value) == list:
                    value_bullets = [f"- {item}" for item in value]
                    value = "\n".join(value_bullets)
                    markdown += f"{value}\n"
                else:
                    markdown += f"{response_dict[key]}\n"
            return markdown
        elif response_format == "dto":
            return response
        raise ValueError("Invalid response_format. Use 'json', 'markdown' or 'dto'.")

    def _augment_prd(self, prd: PRD) -> PRD:
        """Augments the PRD with greater details."""
        prd_json = prd.model_dump()
        print("Augmenting PRD: ", prd_json)
        keys = prd_json.keys()
        for key in keys:
            if key == "title":
                continue
            value = prd_json[key]
            prompt = f"Provide more details about the {key} for the project."
            if type(value) == str:
                prompt += f" Current value: {value}"
                expanded_value = llm(prompt, response_model=str)
                prd_json[key] = expanded_value
            elif type(value) == list:
                expanded_values = []
                for item in value:
                    prompt += f" Current value: {item}"
                    expanded_item = llm(prompt, response_model=str)
                    expanded_values.append(expanded_item)
                prd_json[key] = expanded_values
        print("Augmented PRD: ", prd_json)
        return PRD(**prd_json)

    def get_user_stories(self) -> Union[UserStories, LLMErrorResponse]:
        """Gets the User Stories for the project"""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create User Stories for their projects.
                """
        prompt = f"""
                Provide User Stories for the project with the following details:
                Details: {self.project_description}
                PRD: {self.get_prd()}
                """
        response = llm(prompt=prompt, response_model=UserStories, system=system)
        return response

In [7]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)


def save_markdown_file(markdown: str, file_path: str) -> None:
    """Saves the generated markdown file to the specified path."""
    with open(file_path, "w") as file:
        file.write(markdown)

In [8]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = (
    "A web application for tracking investigating behavioural change in aliens."
)
output_file = "alien_behavioural_change_prd.md"
augment = True

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
prd = minipm.get_prd(response_format="markdown", augment=augment)

save_markdown_file(prd, output_file)

## Render the PRD as markdown
render_output(prd)

Augmenting PRD:  {'title': 'Alien Behavioural Change Tracker', 'overview': 'A web application designed to track and investigate behavioural changes in aliens, providing valuable insights for intergalactic research and cooperation.', 'problem_statement': 'Current methods for tracking alien behaviour are manual, time-consuming, and prone to errors, hindering our understanding of their social and cultural development.', 'business_domains': ['Intergalactic Research', 'Alien Behavioural Studies', 'Space Exploration'], 'proposed_solution': 'Develop a user-friendly, data-driven web application to monitor and analyze alien behaviour, enabling researchers to identify patterns, trends, and correlations.', 'target_audience': 'Intergalactic researchers, astrobiologists, and alien behaviourists', 'features': ['User registration and login', 'Alien profile creation and management', 'Behavioural data collection and analysis', 'Real-time tracking and notifications', 'Data visualization and reporting', 

# Product Requirements Document (PRD) for Alien Behavioural Change Tracker
## Overview
The project, titled 'Galactic Behavioural Insights,' aims to develop a comprehensive web application for monitoring and analyzing behavioural patterns in extraterrestrial life forms. By leveraging advanced data analytics and machine learning algorithms, the platform will enable researchers to identify trends, correlations, and anomalies in alien behaviour, ultimately facilitating a deeper understanding of their social structures, communication methods, and cultural norms. This breakthrough technology will pave the way for enhanced intergalactic cooperation, mutual respect, and peaceful coexistence among diverse cosmic civilizations.
## Problem Statement
The current methods for tracking alien behaviour are manual, time-consuming, and prone to errors. This hinders our understanding of their social and cultural development. To improve this, a more efficient and accurate system is required. Key aspects to focus on include: developing automated data collection techniques, improving data analysis algorithms, and creating a comprehensive database to store and compare findings over time. These advancements would significantly enhance our knowledge of alien societies and cultures.
## Business Domains
- The business domain for the project, Intergalactic Research, involves the study and exploration of the universe beyond Earth's atmosphere. This can include various sub-domains such as astrobiology, exoplanetary science, space technology, and interstellar exploration. Key areas of focus may encompass the search for extraterrestrial life, understanding the formation and evolution of galaxies, and developing advanced propulsion systems for deep space travel.
- The business domains for the project are focused on two primary areas: Intergalactic Research and Alien Behavioural Studies. Intergalactic Research encompasses a broad range of topics including astrobiology, exoplanetary science, and the search for extraterrestrial intelligence (SETI). Alien Behavioural Studies, on the other hand, delves into the social structures, communication methods, and cultural aspects of potential extraterrestrial life forms, aiming to understand their behaviors, habitats, and possible interactions with Earth-based life.
- The business domains for the project encompass a wide range of interdisciplinary fields. Key areas include: 
   1. **Intergalactic Research**: This domain focuses on the systematic study of the universe beyond Earth's atmosphere, including stars, galaxies, and other celestial bodies. 
   2. **Alien Behavioural Studies**: This area of research explores the hypothesized behaviors and social structures of extraterrestrial life forms, considering both theoretical models and potential empirical evidence.
   3. **Space Exploration**: This domain is centered on the physical and technological aspects of traveling through space, with an emphasis on designing missions, developing necessary hardware and software, and conducting operations in space.
## Proposed Solution
The proposed solution, 'Alien Behaviour Monitoring and Analysis Web Application', aims to provide researchers with a comprehensive tool to study and understand alien behaviour. Key features of the application will include: 
   1. Data Collection and Integration: Gathering data from various sources, including sensor readings, observations, and existing research.
   2. Data Visualization: Utilizing interactive and dynamic visualizations to represent complex data, facilitating pattern recognition and trend analysis.
   3. Predictive Analytics: Implementing machine learning algorithms to forecast future alien behaviour, based on historical data and trends.
   4. Collaboration Tools: Enabling researchers to share findings, discuss observations, and work together in real-time.
   5. Customizable Dashboards: Allowing users to create personalized dashboards, focusing on specific aspects of alien behaviour, such as habitat, migration patterns, or social interactions.
   The application will be built using a microservices architecture, ensuring scalability, flexibility, and maintainability. A user-centred design approach will guarantee an intuitive and user-friendly interface, catering to the needs of researchers and scientists.
## Target Audience
The target audience includes a diverse range of intergalactic experts, comprising: 
      1. Intergalactic researchers, focusing on the study of celestial entities and phenomena beyond Earth's atmosphere. 
      2. Astrobiologists, specializing in the discovery and analysis of life forms throughout the universe, 
      3. Alien behaviourists, who examine and interpret the actions, social structures, and communication patterns of extraterrestrial life forms.
## Features
- The project features user registration and login, with the goal of providing a secure and personalized experience. Key aspects include: 
   1. User Registration: allowing new users to create accounts with unique usernames and passwords.
   2. User Login: enabling registered users to access the system with their credentials.
   Future enhancements may include: 
   1. Password Recovery: a mechanism for users to reset forgotten passwords.
   2. Account Management: options for users to edit their profiles and account settings.
   3. Multi-Factor Authentication: adding an extra layer of security to the login process.
- The project features user registration and login functionality, allowing users to create and manage their own accounts. Additionally, it includes alien profile creation and management, enabling users to design and oversee their alien characters. This may involve creating alien profiles with unique characteristics, such as name, species, planet of origin, and special abilities. The management aspect could allow users to edit, update, or delete their alien profiles as needed. Furthermore, the project may include features such as user profile management, where users can view and manage their own account information, and potentially interact with other users' alien profiles, fostering a sense of community and collaboration.
- The project features user registration and login functionality, allowing users to securely create and access their accounts. Additionally, it includes alien profile creation and management, where users can create and manage their alien characters, customizing their attributes, skills, and appearance. The platform also focuses on behavioural data collection and analysis, tracking user interactions, and providing insights into user behavior, preferences, and trends. Further features may include: 
   - User profiling and segmentation 
   - Personalized recommendations 
   - Social sharing and community building 
   - Gamification and incentives 
   - Data visualization and reporting 
   - Integrations with other services and platforms
- The project features user registration and login functionality, allowing users to create and manage their accounts securely. Additionally, it includes alien profile creation and management, enabling users to design and track alien characters. Behavioural data collection and analysis are also key components, providing insights into user interactions and alien behaviour. Real-time tracking and notifications further enhance the user experience, delivering updates and alerts as needed.
- The project features a range of functionalities, including: 
   1. User Registration and Login: allowing users to securely create and manage their accounts, 
   2. Alien Profile Creation and Management: enabling users to generate, edit, and control alien profiles, 
   3. Behavioural Data Collection and Analysis: capturing and interpreting data related to alien behavior, 
   4. Real-time Tracking and Notifications: providing instant updates and alerts regarding alien activity, 
   5. Data Visualization and Reporting: presenting collected data in a clear and comprehensive manner for further evaluation and insights.
- The project features a comprehensive suite of tools, including: 
   1. User registration and login for secure access, 
   2. Alien profile creation and management to store and organize data on alien species, 
   3. Behavioural data collection and analysis to study alien behaviour, 
   4. Real-time tracking and notifications to monitor alien activity, 
   5. Data visualization and reporting to present findings in a clear and concise manner, 
   6. Collaboration tools for research teams to facilitate teamwork and knowledge sharing.
## Constraints
- To ensure data accuracy and validity, the project will implement the following constraints: data type checks, range checks, mandatory field checks, and validation against predefined business rules. Additionally, data will be verified through cross-checks and audits to guarantee its correctness and consistency.
- The project constraints include ensuring data accuracy and validity by implementing robust data validation and verification processes, as well as addressing potential cultural and linguistic barriers through the use of diverse and inclusive language, and providing multilingual support to cater to a wide range of audiences.
- The project constraints include ensuring data accuracy and validity by implementing robust data validation and verification protocols. Additionally, addressing potential cultural and linguistic barriers is crucial through the use of multilingual support, cultural sensitivity training, and inclusive communication strategies. Furthermore, complying with intergalactic regulations and ethics standards requires adherence to a set of principles and guidelines that prioritize transparency, accountability, and respect for diverse forms of life and intelligence. This includes obtaining necessary permits, following established protocols for data collection and sharing, and ensuring that all project activities align with the principles of cooperation, mutual respect, and peaceful coexistence.
- To ensure the success of the project, several key constraints must be considered. Firstly, ensuring data accuracy and validity is crucial, as incorrect or misleading information can have significant consequences. This involves implementing rigorous data validation and verification processes to guarantee the precision and reliability of the data. Secondly, addressing potential cultural and linguistic barriers is essential to ensure that the project is accessible and effective for diverse populations. This may involve providing multilingual support, culturally sensitive interfaces, and adaptability to varying cultural norms. Furthermore, complying with intergalactic regulations and ethics standards is vital to prevent any adverse impacts on the environment, societies, or individuals. This includes adhering to principles such as transparency, accountability, and respect for privacy. Lastly, managing large datasets and scalability is critical to accommodate growing volumes of data and user demands. This involves developing flexible and efficient data management systems, leveraging technologies like cloud computing, and optimizing database performance to ensure seamless data handling and analysis.
## Assumptions
- Assumptions for the project include that alien subjects will not only cooperate but also participate fully in the study, providing accurate and reliable data. This assumption is based on the premise that the extraterrestrial life forms are capable of understanding and communicating effectively with human researchers, and that they are willing to engage in the scientific process without any external influence or coercion. Furthermore, it is assumed that the alien subjects will maintain their cooperation throughout the duration of the study, adhering to the established protocols and procedures. These assumptions are crucial for the validity and reliability of the study's findings, as they directly impact the quality of the data collected and the conclusions drawn from it.
- The project assumptions include the cooperation and participation of alien subjects in the study, as well as access to reliable and consistent data sources. These assumptions are critical to the project's success, as they enable the collection of high-quality data and facilitate meaningful analysis and insights. The project team will need to establish trust with the alien subjects, ensure their safety and well-being, and provide clear explanations of the study's purpose and procedures to maintain their cooperation. Additionally, the team will need to verify the accuracy and reliability of the data sources, and implement robust data validation and quality control measures to ensure the integrity of the data. By doing so, the project can increase the validity and generalizability of its findings, and provide valuable contributions to the field of study.
- Assumptions for the project include: (1) Alien subjects will cooperate and participate in the study, which is a crucial element as it ensures the collection of meaningful data. (2) Access to reliable and consistent data sources is vital for the success of the project, as it allows researchers to draw accurate conclusions. (3) Researchers will have necessary training and expertise to handle the complexities of the study, including the potential for unforeseen challenges related to the alien subjects or the data analysis process.
## Dependencies
- The project dependencies include Intergalactic database integration, which relies on the following components: 
   1. Galactic Data Connector (GDC) for secure data transmission, 
   2. Cosmic Database Driver (CDD) for seamless database interactions, 
   3. Stellar Data Mapper (SDM) for efficient data mapping and transformation, 
   4. Planetary Data Logger (PDL) for real-time logging and error tracking, 
   5. Nebula Data Security Framework (NDSF) for robust data encryption and access control.
- The project's dependencies include integration with the Intergalactic database, which provides access to a vast amount of cosmic data, and Advanced data analytics software, enabling the team to process and analyze complex information from various sources. Additionally, the project requires expertise in data modeling, SQL programming, and data visualization to effectively utilize these dependencies.
- The project relies on several key dependencies to function effectively. These include: 
   1. Intergalactic database integration for seamless data exchange and storage across the galaxy. 
   2. Advanced data analytics software to process and analyze the vast amounts of data collected. 
   3. Secure and reliable cloud infrastructure to ensure redundancy, scalability, and security of the system. 
   Additional dependencies may be considered, such as artificial intelligence and machine learning algorithms for predictive modeling and automation, 
   as well as robust cybersecurity measures to protect against potential threats.
## User Personas
- Dr. Zorvath is a renowned expert in the field of xenopsychology, with a Ph.D. in Alien Behaviour and Cognition. As the lead researcher, Dr. Zorvath has dedicated their career to studying the complex social structures and communication patterns of various extraterrestrial species. With a background in psychology, neuroscience, and astrobiology, Dr. Zorvath brings a unique interdisciplinary approach to the project. Their research focuses on understanding the cognitive and emotional processes that drive alien behaviour, with the ultimate goal of developing effective strategies for interspecies communication and cooperation.
- Two user personas have been identified for the project: Dr. Zorvath, the lead researcher on alien behaviour, and Xexu, an astrobiologist studying alien social structures. Dr. Zorvath's expertise lies in understanding the individual and collective behaviour of alien species, with a focus on their habitats, social interactions, and communication methods. Xexu, on the other hand, is an expert in the social structures of alien societies, analyzing their organization, hierarchies, and cultural norms. Both personas will play crucial roles in informing the project's approach to understanding and interacting with alien life forms.
- The project involves three key user personas: Dr. Zorvath, Xexu, and Thorvath. Dr. Zorvath is the lead researcher on alien behaviour, responsible for overseeing the research and development of the project. Xexu is an astrobiologist studying alien social structures, providing valuable insights into the behaviour and interactions of alien species. Thorvath is the IT specialist responsible for the maintenance and upkeep of the application, ensuring that the technology used in the project is functioning correctly and efficiently.
## Conclusion
The Alien Behavioural Change Tracker web application has the potential to revolutionize our understanding of alien behaviour, enabling more effective cooperation and research in the galaxy. By leveraging advanced data analytics and machine learning algorithms, this platform can provide unprecedented insights into the complexities of alien behaviour, facilitating a new era of intergalactic collaboration and discovery. The successful implementation of this project can lead to a profound impact on the field of astrobiology and beyond, ultimately enhancing humanity's position within the cosmic community.


In [27]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking fitness goals and workouts."

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
user_stories = minipm.get_user_stories()

## Render the PRD as markdown
print(user_stories.model_dump_json(indent=4))

{
    "user_stories": [
        {
            "title": "User Profile Creation",
            "description": "As a user, I want to be able to create and manage my profile so that I can personalize my experience on the platform.",
            "acceptance_criteria": [
                "Users can create a new profile",
                "Users can edit their existing profile",
                "Profiles include basic user information such as name, email, and password"
            ],
            "story_points": 8
        },
        {
            "title": "Goal Setting and Tracking",
            "description": "As a user, I want to be able to set and track my fitness goals so that I can stay motivated and focused on my progress.",
            "acceptance_criteria": [
                "Users can set new fitness goals",
                "Users can view their current goals",
                "Users can track their progress towards their goals"
            ],
            "story_points": 10
        },
  